Давайте разберемся как файнтюнить StyleGAN3 под нашу задачу.

Сперва установим правильную версию PyTorch (на которой был написан StyleGAN):

In [ ]:
import torch
print(torch.__version__)
!pip uninstall torch -y
!pip install torch==1.9.1

import torch
print(torch.__version__)

1.12.0+cu113
Found existing installation: torch 1.9.1+cu111
Uninstalling torch-1.9.1+cu111:
  Successfully uninstalled torch-1.9.1+cu111
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.9.1-cp37-cp37m-manylinux1_x86_64.whl (831.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.9.1 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.9.1 which is incompatible.


1.12.0+cu113


Импортируем на локальный диск репозиторий со стайлганом:

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git

fatal: destination path 'stylegan3' already exists and is not an empty directory.


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Готовим датасет с покемонами:

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

!git clone https://github.com/Sitnich/ssyp-2022-materials

!unzip ssyp-2022-materials/data/pokemon.zip -d pokemon/

In [ ]:
# Создаем датасет специально для дообучения StyleGAN:
# изменяем размер
!python stylegan3/dataset_tool.py --source pokemon/official-jpg-named --dest train/ --resolution=256x256

In [ ]:
%tensorflow_version 1.x
!pip install --upgrade torch==1.9.1+cu111 torchvision==0.10.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install ninja

In [ ]:
# Проводим дообучение модели (из точки с флага --resume)
!python stylegan3/train.py --outdir=training-runs/ --cfg=stylegan2 --data=train/ --freezed=13 \
      --gpus=8 --batch=64 --kimg=100 --gamma=1 --mirror=1 --aug=noaug --cbase=16384 --glr=0.0025 --dlr=0.0025 --mbstd-group=8\
      --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan2/versions/1/files/stylegan2-lsundog-256x256.pkl

In [ ]:
# Генерируем картинки, с помощью файла .pkl дообученной модели
!python stylegan3/gen_images.py --outdir=out/ --trunc=1 --seeds=2 \
    --network=training-runs/00007-stylegan2--gpus1-batch32-gamma0.5/network-snapshot-000001.pkl